Import all necessary packages & libraries

In [1]:
import pandas as pd
import os

Get parent working directory

In [2]:
pwd = os.getcwd()

Save the location of where the `data` directory is

In [3]:
data_dir = 'data'
data_path = os.path.join(os.path.dirname(pwd), data_dir)
data_path

'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data'

Store the names of all parking ticket csv files into a list

In [4]:
raw_csv_data = []
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith('.csv'):
            raw_csv_data.append(os.path.join(root, file))

raw_csv_data

['c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Tickets_2016.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Tickets_2017.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Tickets_2018.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Tickets_2023.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Ticket_2019.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Ticket_2020.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Ticket_2021.csv',
 'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\data\\Parking_Ticket_2022.csv']

### Data transformation and Cleaning

Load all raw csv data files into one pandas dataframe by:
1. Dropping non-required columns
2. Renaming columns to ensure it is uniform
3. Concatenating the pandas dataframes together

In [5]:
combined_dfs = []

for file in raw_csv_data:
    df = pd.read_csv(file, quotechar='"', skipinitialspace=True)

    columns_to_drop = ['FID', 'ObjectId']

    for col in columns_to_drop:
        if col in df.columns:
            df = df.drop(col, axis=1)

    columns_to_rename = {'VIODESCRIPTION': 'violation_description', 'VOIDESCRIPTION':'violation_description', 'LOCATIONDESC1': 'violation_location','ISSUEDATE': 'issue_date', 'ISSUETIME': 'issue_time'}

    df = df.rename(columns=columns_to_rename)

    combined_dfs.append(df)


combined_df = pd.concat(combined_dfs)
combined_df

,violation_description,violation_location,issue_date,issue_time
0,PARKING VEHICLE ON PRIVATE PROPERTY WITHOUT OW...,100 CITY CENTRE DR,1/1/2016,17:37
1,PARKING VEHICLE AT EXPIRED METER,NR 6 BRANT,1/1/2016,07:18
2,FAILING TO OBEY INSTRUCTIONS DISPLAYED ON TRAF...,5990 AIRPORT RD,1/1/2016,07:58
3,PARKING OUTSIDE A PARKING AREA,6145 VISCOUNT RD,1/1/2016,07:24
4,PARKING IN A RESERVED PARKING AREA WITHOUT A ...,6145 VISCOUNT RD,1/1/2016,12:45
...,...,...,...,...
158667,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",55 GLENN HAWTHORNE BLVD,9/9/2022,9:58
158668,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",3025 PEPPER MILL CRT,9/9/2022,9:58
158669,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",40 ANNAGEM BLVD,9/9/2022,9:58
158670,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",1500 DUNDAS ST E,9/9/2022,9:58


1. Ensure all text in the `violation_description` column are all upper case
2. Change `issue_date` format to standard pandas date format
3. Change `issue_time` format to standard pandas time formt

In [6]:
cleaned_df = combined_df.copy()
cleaned_df['violation_description'] = cleaned_df['violation_description'].str.upper()
cleaned_df['issue_date'] = pd.to_datetime(cleaned_df['issue_date'])
cleaned_df['issue_time'] = pd.to_datetime(cleaned_df['issue_time'], format='%H:%M').dt.time

cleaned_df

,violation_description,violation_location,issue_date,issue_time
0,PARKING VEHICLE ON PRIVATE PROPERTY WITHOUT OW...,100 CITY CENTRE DR,2016-01-01,17:37:00
1,PARKING VEHICLE AT EXPIRED METER,NR 6 BRANT,2016-01-01,07:18:00
2,FAILING TO OBEY INSTRUCTIONS DISPLAYED ON TRAF...,5990 AIRPORT RD,2016-01-01,07:58:00
3,PARKING OUTSIDE A PARKING AREA,6145 VISCOUNT RD,2016-01-01,07:24:00
4,PARKING IN A RESERVED PARKING AREA WITHOUT A ...,6145 VISCOUNT RD,2016-01-01,12:45:00
...,...,...,...,...
158667,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",55 GLENN HAWTHORNE BLVD,2022-09-09,09:58:00
158668,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",3025 PEPPER MILL CRT,2022-09-09,09:58:00
158669,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",40 ANNAGEM BLVD,2022-09-09,09:58:00
158670,"PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...",1500 DUNDAS ST E,2022-09-09,09:58:00


### Exporting data

Save the location of the directory where the cleaned data will be stored

In [7]:
cleaned_data_dir = 'cleaned-data'
cleaned_data_path = os.path.join(os.path.dirname(pwd), cleaned_data_dir)
cleaned_data_path

'c:\\Users\\arzek\\Desktop\\repo\\mississauga-parking-violations-data-analysis\\cleaned-data'

Save the pandas dataframe with the combined csv files into one single giant csv file

In [8]:
cleaned_df.to_csv(f'{cleaned_data_path}\\combined_mississauga_parking_ticket_dataset.csv', index=False)

Saving the pandas dataframe with the combined csv files into a single SQLite Database

In [14]:
import sqlite3

conn = sqlite3.connect(f'{cleaned_data_path}\\mississauga_parking_tickets_dataset.db')

cleaned_df.to_sql('parking_violations', conn, if_exists='replace', index=False)

conn.close()

Verify the data exists in the database

In [15]:
# Reconnect to the database
conn = sqlite3.connect(f'{cleaned_data_path}\\mississauga_parking_tickets_dataset.db')

# Query the database to see the saved data
df_from_db = pd.read_sql_query("SELECT * FROM parking_violations", conn)

# Display the DataFrame loaded from the database
print(df_from_db)

# Close the connection again
conn.close()

                                     violation_description  \
0        PARKING VEHICLE ON PRIVATE PROPERTY WITHOUT OW...   
1                         PARKING VEHICLE AT EXPIRED METER   
2        FAILING TO OBEY INSTRUCTIONS DISPLAYED ON TRAF...   
3                           PARKING OUTSIDE A PARKING AREA   
4        PARKING IN A  RESERVED PARKING AREA WITHOUT A ...   
...                                                    ...   
1316045  PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...   
1316046  PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...   
1316047  PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...   
1316048  PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...   
1316049  PARK, STOP OR LEAVE VEHICLE IN AN ACCESSIBLE P...   

              violation_location           issue_date       issue_time  
0             100 CITY CENTRE DR  2016-01-01 00:00:00  17:37:00.000000  
1                     NR 6 BRANT  2016-01-01 00:00:00  07:18:00.000000  
2                5990 AIRPORT RD  20